In [0]:
%pip install -U -qqq langchain_core langchain_databricks langchain_community
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_databricks import ChatDatabricks
from databricks.sdk import WorkspaceClient
import os

# configure workspace tokens
w = WorkspaceClient()
os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

def format_context(df: pd.DataFrame) -> str:
    """
    Converts the DataFrame into a JSON string to ensure all data is passed
    to the model without truncation. JSON is also a great format for structured data
    like you have in 'description_by_sections'.
    """
    return df.to_json(orient='records', indent=3)



@tool
def find_places_by_keyword(search_term: str) -> str:
  """
  Searches the google maps place daily for finding places that are family friendly
  """
  query = f"""
    SELECT
    title, zip_code, city, country FROM 
     `hackathon-catalog`.`nimble`.`dbx_google_maps_place_daily`
   WHERE TO_JSON(accessibility) LIKE '%{search_term}%'
    LIMIT 5
  """
  print(f"Executing query: {query}") # For debugging
  df = spark.sql(query).toPandas()
  return df.to_json(orient='records', indent=2)

@tool
def find_countries_by_keyword(country_name: str) -> str:
  """
  Searches the google maps place daily for finding places that are located in US or AU
  The input must be a single string. For example: "US"
  """
  query = f"""
    SELECT
    title, zip_code, city, country FROM 
     `hackathon-catalog`.`nimble`.`dbx_google_maps_place_daily`
   WHERE country LIKE '%{country_name}%'
    LIMIT 5
  """
  print(f"Executing query: {query}") # For debugging
  df = spark.sql(query).toPandas()
  return df.to_json(orient='records', indent=2)  

  print(f"Executing query: {query}") # For debugging
  df = spark.sql(query).toPandas()
  return df.to_json(orient='records', indent=2)  

  w = WorkspaceClient()
os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

# The LLM is the "brain" of the agent
llm = ChatDatabricks(endpoint="databricks-llama-4-maverick", max_tokens=4048)

# The Agent needs a list of tools it can use
tools = [find_countries_by_keyword, find_places_by_keyword]

# This prompt template tells the agent how to reason and use tools
prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}
Use the `find_countries_by_keyword` tool for the country.
Then, use the `find_places_by_keyword` tool for the keyword
After you have gathered all the places information from your searches, combine the results into a single, helpful summary for the user.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: After you find the places from all your searches, your final answer should summarize and list them
Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

# Create the agent by combining the LLM, tools, and prompt
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Let's ask our agent to find a product related to accessibility
response = agent_executor.invoke({
    "input": "I am looking for places that are family friendly in US country"
})

print("\n\n--- Final Answer ---")
print(response['output'])
     



> Entering new AgentExecutor chain...
To find family-friendly places in the US, I first need to confirm that "US" refers to the United States, which is a valid country. Then, I will search for family-friendly places within that country.

Action: find_countries_by_keyword
Action Input: "US"Executing query: 
    SELECT
    title, zip_code, city, country FROM 
     `hackathon-catalog`.`nimble`.`dbx_google_maps_place_daily`
   WHERE country LIKE '%US%'
    LIMIT 5
  
[
  {
    "title":"13 Market St",
    "zip_code":"28401",
    "city":"Wilmington",
    "country":"US"
  },
  {
    "title":"Sports Page",
    "zip_code":"94043",
    "city":"Mountain View",
    "country":"US"
  },
  {
    "title":"Empire State Building",
    "zip_code":"10001",
    "city":"New York",
    "country":"US"
  },
  {
    "title":"Levain Bakery \u2013 NoHo, NYC",
    "zip_code":"10012",
    "city":"New York",
    "country":"US"
  },
  {
    "title":"Milano's Bar",
    "zip_code":"10012",
    "city":"New York",
    